Influence of remeshing on the result quality
====================================

In [ ]:
import sys
import os
sys.path.append(os.path.realpath(".."))
from preamble import *
from io_ply import read_ply, write_ply
from geometry import massmatrix_voronoi

fontsize = 18
recompute = True

In [ ]:
# Compute voronoi cell areas
masses = []
for i, method in enumerate(["reg", "base", "remesh_start", "remesh_middle"]):
    mesh = read_ply(os.path.join(basename, f"res_{method}.ply"))
    v, f = mesh["vertices"], mesh["faces"].long()
    masses.append(massmatrix_voronoi(v, f).cpu().numpy())

vmax = max([np.max(M) for M in masses])
vmin = min([np.min(M) for M in masses])
norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)

for i, method in enumerate(["reg", "base", "remesh_start", "remesh_middle"]):
    mesh = read_ply(os.path.join(basename, f"res_{method}.ply"))
    v, f = mesh["vertices"].cpu().numpy(), mesh["faces"].cpu().numpy()
    c = matplotlib.cm.magma_r(norm(masses[i]))[:,:3]
    write_ply(os.path.join(basename, f"res_{method}.ply"), v, f, vc=c, ascii=True)

In [ ]:
# Load wireframes
wireframes = []
res = 100
for i, method in enumerate(["reg", "base", "remesh_start", "remesh_middle"]):
    filename = os.path.join(basename, f"res_{method}_wireframe.png")
    if not os.path.exists(filename) or recompute:
        blender_render(os.path.join(basename, f"res_{method}.ply"), 
                       basename, 
                       25,
                       8,
                       res=res,
                       wireframe=True,
                       area = True)
    wireframes.append(plt.imread(filename))

In [ ]:
base_size = 3
n_rows = 2
n_cols = 2
H,W,_ = wireframes[0].shape
w = H
h = H-150
aspect = w/h
inset_w = 0.2*w
fig = plt.figure(1, figsize=(n_cols * base_size* aspect, n_rows * base_size), constrained_layout=True)
gs = fig.add_gridspec(n_rows,  n_cols, wspace=0.07,hspace=0.0)

titles = ["(a) Regularized", "(b) Ours, no remeshing", "(c) Ours, denser initial shape", "(d) Ours, with remeshing"]

sns.set_style('white')
for i in range(n_rows):
    for j in range(n_cols):
        idx = i*n_rows + j
        ax = fig.add_subplot(gs[i,j])
        ax.tick_params(
            axis='both',
            which='both',
            labelleft=False,
            labelbottom=False)
        im_cropped = np.flip(wireframes[idx][150:, (W-w)//2:(W+w)//2], axis=0)
        ax.imshow(im_cropped, origin='lower', interpolation='none')
        axins = ax.inset_axes([-0.3, 0, 0.5, 0.5])

        axins.imshow(im_cropped, origin='lower', interpolation='none')
        axins.tick_params(axis='both', which='both', labelleft=False, labelbottom=False)
        x0 = int(w*0.3)
        y0 = int(0.21*H)
        x1, x2, y1, y2 = (x0, x0+inset_w, y0, y0+inset_w)
        axins.set_xlim(x1, x2)
        axins.set_ylim(y1, y2)
        ax.indicate_inset_zoom(axins, edgecolor="black", alpha=1)
        sns.despine(ax=ax, left=True, bottom=True)
        ax.set_xlabel(titles[idx], fontsize=fontsize)

cbar = fig.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=matplotlib.cm.magma_r),
             ax=fig.axes, orientation="horizontal", shrink=0.8)
cbar.ax.set_xlabel("Voronoi cell area", fontsize=fontsize)
cbar.ax.xaxis.set_ticks_position('top')

plt.savefig(os.path.join(basename, "remeshing.pdf"), format='pdf', dpi=300, bbox_inches='tight', pad_inches=0.03)